In [54]:
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "seaborn"

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.patches import Rectangle

import sys
sys.path.append("..")

from pygalaxy import Euler_symplectic_tree, ADB6, RK4, Stormer_verlet, Optimized_815
#from pygalaxy import init_solar_system
from pygalaxy.barnes_hut_array import compute_energy_and_tree_structure
#from pygalaxy.naive import compute_energy

In [55]:
def init_solar_system(dim=2):
    """ Initialize the solar system with particles in d dimensions.

    Args:
        dim: Number of dimensions (default is 3).

    Returns:
        mass: Array of masses.
        bodies: Array of particles [nb_particles, dim, x], where x includes position and velocity.
    """
    assert( (dim>=2)&(dim<=3))
    # Initial positions and velocities
    if dim == 2:
        positions = np.array([[        0, 0],       # Sun
                            [    -46e9, 0],       # Mercury
                            [ -10748e7, 0],       # Venus
                            [-147095e6, 0],       # Earth
                            [ -20662e7, 0]])      # Mars

        velocities = np.array([[      0,      0],   # Sun
                            [      0, -58980],    # Mercury
                            [      0, -35260],    # Venus
                            [      0, -30300],    # Earth
                            [      0, -26500]])   # Mars
    elif dim == 3:
        positions = np.array([[        0, 0, 0],       # Sun
                            [    -46e9, 0, 0],       # Mercury
                            [ -10748e7, 0, 0],       # Venus
                            [-147095e6, 0, 0],       # Earth
                            [ -20662e7, 0, 0]])      # Mars

        velocities = np.array([[      0,      0, 1000],   # Sun
                            [      0, -58980, 0],    # Mercury
                            [      0, -35260, 0],    # Venus
                            [      0, -30300, 0],    # Earth 
                            [      0, -26500, 0]])   # Mars
    else:
        return ValueError

    # Combine positions and velocities
    bodies = np.empty((positions.shape[0], dim, 2))  # [nb_particles, dim, 2]
    bodies[:, :, 0] = positions
    bodies[:, :, 1] = velocities

    # Masses of the bodies
    mass = np.array([  1.989e30,  # Sun
                      0.33011e24, # Mercury
                      4.8675e24,  # Venus
                       5.972e24,  # Earth
                      6.4171e23]) # Mars
    planets = ['sun', 'mercury', 'venus', 'earth', 'mars', 'jupiter', 
           'saturn', 'uranus', 'neptune']

    return mass, bodies, planets[:mass.shape[0]]

In [56]:
# Simulation parameters
dt = 86400  # 1 day in seconds
nt = 1000  # Number of time steps
dim = 3  # Number of dimensions

# Initialize solar system
mass, particles, planets = init_solar_system(dim)

# Number of bodies
nbodies = particles.shape[0]

# Initialize the integrator
time_method = Euler_symplectic_tree(dt, nbodies, dim, compute_energy_and_tree_structure)  # Replace `None` with the appropriate force computation method

# Arrays to store positions and velocities over time
coords = np.zeros((nt, nbodies, dim))
coords[0] = particles[:, :, 0]  # Initial positions

velocities = np.zeros((nt, nbodies, dim))
velocities[0] = particles[:, :, 1]  # Initial velocities

#trees = []

# Time-stepping loop
for i in range(nt - 1):
    time_method.update(mass, particles)
    coords[i + 1] = particles[:, :, 0]
    velocities[i + 1] = particles[:, :, 1]

In [57]:
fig = go.Figure()

for i in range(5):
    if i==0: mode='markers'
    else: mode='lines'
    fig.add_trace(go.Scattergl(x=coords[:,i,0], y=coords[:,i,1], mode=mode, name=planets[i]))
    #fig.add_trace(go.Scattergl(x=[coords[-1,i,0]], y=[coords[-1,i,1]], mode='markers', showlegend=False, marker_color='grey'))


fig.update_layout(title='Solar system XY')
fig.update_xaxes(title='x', exponentformat='e')
fig.update_yaxes(title='y', exponentformat='e')
fig.show()

In [58]:
fig = go.Figure()

for i in range(5):
    if i==0: mode='markers'
    else: mode='lines'
    fig.add_trace(go.Scattergl(x=coords[:,i,0], y=coords[:,i,2], mode=mode, name=planets[i]))
    #fig.add_trace(go.Scattergl(x=[coords[-1,i,0]], y=[coords[-1,i,1]], mode='markers', showlegend=False, marker_color='grey'))


fig.update_layout(title='Solar system XZ')
fig.update_xaxes(title='x', exponentformat='e')
fig.update_yaxes(title='z', exponentformat='e')
fig.show()

In [59]:
fig = go.Figure()

for i in range(5):
    if i==0: mode='markers'
    else: mode='lines'
    fig.add_trace(go.Scattergl(x=coords[:,i,1], y=coords[:,i,2], mode=mode, name=planets[i]))
    #fig.add_trace(go.Scattergl(x=[coords[-1,i,0]], y=[coords[-1,i,1]], mode='markers', showlegend=False, marker_color='grey'))


fig.update_layout(title='Solar system YZ')
fig.update_xaxes(title='y', exponentformat='e')
fig.update_yaxes(title='z', exponentformat='e')
fig.show()